# Notebook magique with AutoML

In [4]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sns as sns







from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack




ModuleNotFoundError: No module named 'sns'